<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/01.%20Using_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-intro} -->

# Understanding LLM APIs

We will explore OpenAI models API to generate text.

<!--- @wandbcode{llmapps-intro} -->

### Setup

In [1]:
%pip install --upgrade openai tiktoken wandb -qq

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai
import tiktoken
import wandb
from pprint import pprint
from getpass import getpass
from wandb.integration.openai import autolog

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [9]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [12]:
if True:
  print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Please enter password in the VS Code prompt at the top of your VS Code window!
OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [14]:
# start logging to W&B
autolog({"project":"llmapps", "entity":"cmagganas", "job_type": "introduction"})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: cmagganas. Use `wandb login --relogin` to force relogin


# Tokenization

In [15]:
encoding = tiktoken.encoding_for_model("text-davinci-003")
enc = encoding.encode("Weights & Biases is awesome!")
print(enc)
print(encoding.decode(enc))

[1135, 2337, 1222, 8436, 1386, 318, 7427, 0]
Weights & Biases is awesome!


we can decode the tokens one by one

In [16]:
for token_id in enc:
    print(f"{token_id}\t{encoding.decode([token_id])}")

1135	We
2337	ights
1222	 &
8436	 Bi
1386	ases
318	 is
7427	 awesome
0	!


> Note how the leading tokens contain spacing.

# Sampling

Let's sample some text from the model. For this, let's create a wrapper function around the temperature parameters.
Higher temperature will result in more random samples.

In [17]:
def generate_with_temperature(temp):
  "Generate text with a given temperature, higher temperature means more randomness"
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    temperature=temp,
  )
  return response.choices[0].text

In [18]:
for temp in [0, 0.5, 1, 1.5, 2]:
  pprint(f'TEMP: {temp}, GENERATION: {generate_with_temperature(temp)}')

('TEMP: 0, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and visualizing machine '
 'learning experiments. It helps to keep track of all the different '
 'experiments you run, and provides powerful visualizations to help you '
 "understand the results. It's a great way")
('TEMP: 0.5, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and managing machine '
 'learning experiments. It allows you to easily visualize, compare, and share '
 'the results of your experiments, helping you to identify trends and improve '
 'your models. With its powerful search')
('TEMP: 1, GENERATION: \n'
 '\n'
 'Weights & Biases is an AI experiment tracking platform that enables data '
 'scientists and ML engineers to track and visualize the performance of their '
 "machine learning models. It's a great tool for tracking, analyzing, and "
 'comparing ML experiments, and for creating')
('TEMP: 1.5, GENERATION: \n'
 '\n'
 'Weights & Biases is a powerful tool that

You can also use the [`top_p` parameter](https://platform.openai.com/docs/api-reference/completions/create#completions/create-top_p) to control the diversity of the generated text. This parameter controls the cumulative probability of the next token. For example, if `top_p=0.9`, the model will pick the next token from the top 90% most likely tokens. The higher the `top_p` the more likely the model will pick a token that it hasn't seen before. You should only use one of `temperature` or `top_p` at a given time.

In [19]:
def generate_with_topp(topp):
  "Generate text with a given top-p, higher top-p means more randomness"
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    top_p=topp,
    )
  return response.choices[0].text

In [20]:
for topp in [0.01, 0.1, 0.5, 1]:
  pprint(f'TOP_P: {topp}, GENERATION: {generate_with_topp(topp)}')

('TOP_P: 0.01, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and analyzing machine '
 'learning experiments. It provides powerful visualizations and insights into '
 'model performance, enabling data scientists to quickly identify areas of '
 'improvement and optimize their models.')
('TOP_P: 0.1, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and analyzing machine '
 'learning experiments. It provides powerful visualizations and insights into '
 'model performance, enabling data scientists to quickly identify areas of '
 'improvement and optimize their models.')
('TOP_P: 0.5, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and analyzing machine '
 'learning experiments. It helps to easily compare different models and their '
 'performance, and provides useful visualizations and insights to help make '
 'data-driven decisions.')
('TOP_P: 1, GENERATION: \n'
 '\n'
 'Weights & Biases is a powerful machine learni

# Chat API

Let's switch to chat mode and see how the model responds to our messages. We have some control over the model's response by passing a `system-role`, here we can steer to model to adhere to a certain behaviour.

> We are using `gpt-3.5-turbo`, this model is faster and cheaper than `davinci-003`

In [21]:
MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say something about Weights & Biases"},
    ],
    temperature=0,
)

response

<OpenAIObject chat.completion id=chatcmpl-7VuH8dfXQjHMoLx4fGS9VIMAhgFuH at 0x7fa7787a4b80> JSON: {
  "id": "chatcmpl-7VuH8dfXQjHMoLx4fGS9VIMAhgFuH",
  "object": "chat.completion",
  "created": 1687838846,
  "model": "gpt-3.5-turbo-0301",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Weights & Biases is a machine learning platform that helps data scientists and machine learning engineers track and visualize their experiments. It provides tools for experiment management, hyperparameter tuning, and model visualization, making it easier to understand and improve machine learning models. Weights & Biases also offers integrations with popular machine learning frameworks like TensorFlow, PyTorch, and Keras."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 27,
    "completion_tokens": 74,
    "total_tokens": 101
  }
}

As you can see above, the response is a JSON object with relevant information about the request.

In [22]:
pprint(response.choices[0].message.content)

('Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track and visualize their experiments. It '
 'provides tools for experiment management, hyperparameter tuning, and model '
 'visualization, making it easier to understand and improve machine learning '
 'models. Weights & Biases also offers integrations with popular machine '
 'learning frameworks like TensorFlow, PyTorch, and Keras.')


In [23]:
wandb.finish()

usage/completion_tokens,▂▂▂▂▂▁▁▁▂█
usage/elapsed_time,█▆▃▄▆▂▁▅▄▅
usage/prompt_tokens,▁▁▁▁▁▁▁▁▁█
usage/total_tokens,▂▂▂▂▂▁▁▁▂█
usage/completion_tokens,74
usage/elapsed_time,3.59035
usage/prompt_tokens,27
usage/total_tokens,101
